#### 启动一个线程就是把一个函数传入并创建Thread实例，然后调用start()开始执行：

In [2]:
import threading, time

# 线程执行代码
def loop():
    print(f"线程{threading.current_thread().name}正在运行中...")
    n = 1
    while n <= 5:
        n += 1
        print(f"线程 {threading.current_thread().name} >>> {n}")
        time.sleep(1)
    print(f"线程{threading.current_thread().name}结束了")


print(f"线程{threading.current_thread().name}正在运行中...")

t = threading.Thread(target=loop, name="循环线程")
t.start()
t.join()

print(f"线程{threading.current_thread().name}结束了")

线程MainThread正在运行中...
线程循环线程正在运行中...
线程 循环线程 >>> 2
线程 循环线程 >>> 3
线程 循环线程 >>> 4
线程 循环线程 >>> 5
线程 循环线程 >>> 6
线程循环线程结束了
线程MainThread结束了


由于任何进程默认就会启动一个线程，我们把该线程称为主线程，主线程又可以启动新的线程，Python的threading模块有个current_thread()函数，它永远返回当前线程的实例

## Lock
#### 多进程中，每个变量在每个进程中各自有一个拷贝
#### 多线程中，所有变量由所有线程共享

In [12]:
# 看看多线程同时操作对变量的影响
import time, threading

balance = 0  # 总存款


def change_it(n):
    # 现存 后取，结果应该为0
    global balance
    balance += n
    balance -= n


def run_thread(n):
    for _ in range(100):
        change_it(n)


t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
print(t1,type(t1))
print(t2)
t1.start()
t2.start()
t1.join()
t2.join()
print(f"balance = {balance}")

<Thread(Thread-14, initial)> <class 'threading.Thread'>
<Thread(Thread-15, initial)>
balance = 0


两个线程同时一存一取，就可能导致余额不对，你肯定不希望你的银行存款莫名其妙地变成了负数，所以，我们必须确保一个线程在修改balance的时候，别的线程一定不能改。

如果我们要确保balance计算正确，就要给change_it()上一把锁，当某个线程开始执行change_it()时，我们说，该线程因为获得了锁，因此其他线程不能同时执行change_it()，只能等待，直到锁被释放后，获得该锁以后才能改。由于锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突。创建一个锁就是通过threading.Lock()来实现

In [13]:
import threading

balance = 0
lock = threading.Lock()


def change_it(n):
    # 现存 后取，结果应该为0
    global balance
    balance += n
    balance -= n


def run_thread(n):
    for _ in range(1000000):
        # 先要得到锁,锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突
        # 当多个线程同时执行lock.acquire()时，只有一个线程能成功地获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止。
        lock.acquire()
        try:
            # 随意修改
            change_it(n)
        finally:
            # 改完一定要释放锁
            lock.release()


t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(f"balance = {balance}")

balance = 0


锁的坏处: 
* 阻止了多线程并发执行
* 多个线程持有不同的锁，并试图获得对方的锁，可能会造成死锁，导致多个线程全部挂起

In [6]:
import multiprocessing, threading

multiprocessing.cpu_count()

4

In [7]:
import _thread
import time

# _thread创建的线程，在函数调用结束后会悄悄的退出，
# 而且没有threading 线程的 join() 阻塞功能，所以主线程必须人为阻塞，否则效果不明显。


def increase(inc, lock):
    global balance
    # _thread.TIMEOUT_MAX：参数 timeout 的最大值，超过抛出 OverflowError 异常
    lock.acquire(timeout=_thread.TIMEOUT_MAX)

    print(
        "(Locked: %s) Thread is %s..." % (lock.locked(), _thread.get_ident())
    )  # 锁状态 | 线程描述符
    for i in range(10):
        balance += inc
        time.sleep(0.1)
        print(balance)
    lock.release()


print("Mainthread Start")
balance = 0
lock = _thread.allocate_lock()
t1 = _thread.start_new_thread(increase, (1, lock))
t2 = _thread.start_new_thread(increase, (10, lock))
time.sleep(3)
print("Mainthread Finished")

Mainthread Start
(Locked: True) Thread is 123145377230848...
1
2
3
4
5
6
7
8
9
10
(Locked: True) Thread is 123145382486016...
20
30
40
50
60
70
80
90
100
110
Mainthread Finished


0